In [3]:
import os
import cv2
import torch
import torchvision

In [4]:
device = torch.device('cuda:0')

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/tips&tricks/Project/models/state_dict_model.pt'))

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


<All keys matched successfully>

In [8]:
model.to(device)
model.eval();

In [21]:
img_folder = '/content/drive/MyDrive/Datasets/test/video_0/frames'
output_folder = '/content/drive/MyDrive/Datasets/test/video_0/output_0'
img_name_list = os.listdir(img_folder)
img_name_list.sort()
for img_name in img_name_list:
    image = cv2.imread(os.path.join(img_folder, img_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = torch.from_numpy(image.transpose((2, 0, 1)))
    image_tensor = image_tensor.float().div(255)
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        target = model([image_tensor])
        boxes = target[0]['boxes'].tolist()
        for box in boxes:
            cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color=(255, 0, 0), thickness=2)
    filename = os.path.join(output_folder, img_name) 
    cv2.imwrite(filename, image)

In [24]:
def video_packer(path_in, path_out, delete_after=False):
    img_array = []
    for root, dir, files in os.walk(path_in):
        for filename in files:
            img = cv2.imread(os.path.join(root, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img) 
    out = cv2.VideoWriter(os.path.join(path_out, 'project.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [25]:
video_packer('/content/drive/MyDrive/Datasets/test/video_0/output_0', '/content/drive/MyDrive/Datasets/test/video_0',)